Control DataFrame visual size

In [2]:
%%html
<style>
/* Any CSS style can go in here. */
.dataframe th {
    font-size: 20px;
}
.dataframe td {
    font-size: 20px;
}
</style>

In [1]:
import numpy as np
import pandas as pd
from pieces import *
from game_logic import *

To code:

    check forces move out of check
    - switch last move feature to live in board object
        checkmate
        - make sure working
    en pessaant?

    convert to .py and run games in terminal

    move syntax conversion

In [300]:
class piece:

    def __init__(self, color, ) -> None:
        self.color = color

    def __str__(self):
        piece_dict = {'bking':'♔', 
                'bqueen': '♕', 
                'brook': '♖', 
                'bbishop':'♗', 
                'bknight' : '♘', 
                'bpawn':'♙', 
                'wking': '♚',
                'wqueen': '♛',
                'wrook':'♜', 
                'wbishop':'♝', 
                'wknight': '♞',
                'wpawn' : '♟' 
                }
        return piece_dict[self.get_piece_type()]

    def get_piece_type(self):
        return self.color + self.__class__.__name__
    
    def valid_horizontal_moves(self, board, x, y, lim = None, attacks_only=False):
        """Gets valid horizontal moves in left/right direction

        Parameters
        ----------
        self : piece object (rook, queen, or king)
            chess piece moves are evaluated on
        board : board
            chess board object 
        x : int 
            x position of the piece
        y : int
            y position of the piece

        Returns
        -------
        list
            a list of tuples of (x, y) positions of valid moves
        """
        moves = []

        right_lim = 8-x
        left_lim = x-1

        def right_func(x, y, i):
            return x+i, y
        def left_func(x, y, i):
            return x-i, y

        if lim:
            right_lim, left_lim, = lim, lim
        
        moves.extend(self.valid_moves_helper(board, x, y, right_func, right_lim, attacks_only=attacks_only))
        moves.extend(self.valid_moves_helper(board, x, y, left_func, left_lim, attacks_only=attacks_only))

        return moves
    
    def valid_vertical_moves(self, board, x, y, lim = None, attacks_only=False):
        """Gets valid vertical moves in up/down direction

        Parameters
        ----------
        self : piece object (rook, queen, or king)
            chess piece moves are evaluated on
        board : board
            chess board object 
        x : int 
            x position of the piece
        y : int
            y position of the piece

        Returns
        -------
        list
            a list of tuples of (x, y) positions of valid moves
        """
        moves = []

        up_lim = 8-y
        down_lim = y-1

        def up_func(x, y, i):
            return x, y + i
        def down_func(x, y, i):
            return x, y - i
        
        if lim:
            up_lim, down_lim, = lim, lim
        
        moves.extend(self.valid_moves_helper(board, x, y, up_func, up_lim, attacks_only=attacks_only))
        moves.extend(self.valid_moves_helper(board, x, y, down_func, down_lim, attacks_only=attacks_only))

        return moves
    
    def valid_diagonal_moves(self, board, x, y, lim= None, attacks_only=False):
        """Gets valid diagonal moves in left/right direction

        Parameters
        ----------
        self : piece object (bishop, queen, or king)
            chess piece moves are evaluated on
        board : board
            chess board object 
        x : int 
            x position of the piece
        y : int
            y position of the piece

        Returns
        -------
        list
            a list of tuples of (x, y) positions of valid moves
        """
        moves = []

        up_right_lim = min(8-x, 8-y)
        down_right_lim = min(8-x, y-1)
        up_left_lim = min(x-1, 8-y)
        down_left_lim = min(x-1, y-1)

        def up_right_func(x, y, i):
            return x + i, y + i
        def down_right_func(x, y, i):
            return x + i, y - i
        def up_left_func(x, y, i):
            return x - i, y + i
        def down_left_func(x, y, i):
            return x - i, y - i
        
        if lim:
            up_right_lim, down_right_lim, up_left_lim, down_left_lim = lim, lim, lim, lim
        
        moves.extend(self.valid_moves_helper(board, x, y, up_right_func, up_right_lim, attacks_only=attacks_only))
        moves.extend(self.valid_moves_helper(board, x, y, down_right_func, down_right_lim, attacks_only=attacks_only))
        moves.extend(self.valid_moves_helper(board, x, y, up_left_func, up_left_lim, attacks_only=attacks_only))
        moves.extend(self.valid_moves_helper(board, x, y, down_left_func, down_left_lim, attacks_only=attacks_only))

        return moves

    def valid_moves_helper(self, board, x, y, xy_func, lim, attacks_only = None, pawn_take = False):
        """Gets valid moves in a provided direction up to a specified limit

        Parameters
        ----------
        self : piece object (rook, bishop, queen, or king)
            chess piece moves are evaluated on
        board : board
            chess board object 
        x : int 
            x position of the piece
        y : int
            y position of the piece
        xy_func : function
            function that returns the next square location in a direction given the current square
        lim : int
            1 + the limit of squares to check in a given direction  

        Returns
        -------
        list
            a list of tuples of (x, y) positions of valid moves
        """
        moves = []

        block_color = self.color


        for i in range(1, lim+1):
            xi, yi = xy_func(x, y, i)
            new_square = board.get_square(xi, yi)
            if new_square == None:
                break
            # print('Square {},{}'.format(xi, yi))
            
                # if new_square != '' and new_square.color != block_color:
                #     moves.append((xi, yi))
                break
            # print("   ", new_square, block_color)
            if new_square == '':
                if pawn_take == True:
                    if attacks_only == True:
                        moves.append((xi, yi))
                        # print('pawn attack line')
                    break
                # print('empty sq move')
                moves.append((xi, yi))
            elif new_square.color == block_color:
                # print(new_square, pawn_take, attacks_only)
                if attacks_only == True:
                    moves.append((xi, yi))
                    # print('attack line protect')
                break
            else:
                if isinstance(board.get_square(x,y), pawn) and pawn_take == False:
                    # print('pawn')
                    break
                # print('take')
                moves.append((xi, yi))
                break

        return moves



In [96]:
class knight(piece):

    def __init__(self, color) -> None:
        super().__init__(color)

    def find_moves(self, board, x, y, attacks_only=False):
        moves = []

        move_directions = [(1,2),(2,1),(1,-2),(-2,1),(-1,2),(2,-1),(-1,-2),(-2,-1)]

        for move in move_directions:
            x_new, y_new = x+move[0], y+move[1]
            new_square = board.get_square(x_new, y_new)

            if (new_square != None) and ((new_square == '') or ((new_square.color != self.color) or attacks_only)):
                moves.append((x_new, y_new))
        
        return moves
    

class king(piece):

    def __init__(self, color) -> None:
        super().__init__(color)
        self.moved = False

    def find_moves(self, board, x, y, attacks_only = False):
        moves = []

        moves.extend(self.valid_horizontal_moves(board, x, y, lim = 1, attacks_only=attacks_only))
        moves.extend(self.valid_vertical_moves(board, x, y, lim = 1, attacks_only=attacks_only))
        moves.extend(self.valid_diagonal_moves(board, x, y, lim = 1, attacks_only=attacks_only))

        if attacks_only == False:

            if self.color == 'w': 
                attack_color = 'b'
            else: 
                attack_color = 'w'
            squares_in_check = board.get_attacks(attack_color)
            # print('check squares: ', squares_in_check)
            
            # check for castling
            if self.moved == False:
                    
                corner_piece1 = board.get_square(1,y)
                corner_piece8 = board.get_square(8,y)

                if ((isinstance(corner_piece1, rook)) and 
                    (corner_piece1.moved == False) and 
                    (board.get_square(6,y) == '') and 
                    (board.get_square(7,y) == '') and
                    ((6,y) not in squares_in_check) and
                    ((7,y) not in squares_in_check)):
                        moves.append((7,y))

                if ((isinstance(corner_piece8, rook)) and 
                    (corner_piece8.moved == False) and 
                    (board.get_square(3,y) == '') and 
                    (board.get_square(4,y) == '') and
                    (board.get_square(2,y) == '') and
                    ((3,y) not in squares_in_check) and
                    ((4,y) not in squares_in_check)):
                        moves.append((3,y))

            moves = [i for i in moves if i not in squares_in_check]

        # print(moves)


        #### Castling rules ####
        # Neither the king nor the rook has previously moved.
        # There are no pieces between the king and the rook.
        # The king is not currently in check.
        # The king does not pass through or finish on a square that is attacked by an enemy piece.

        return moves
        
class queen(piece):

    def __init__(self, color) -> None:
        super().__init__(color)

    def find_moves(self, board, x, y, attacks_only=False):
        moves = []

        moves.extend(self.valid_horizontal_moves(board, x, y, attacks_only=attacks_only))
        moves.extend(self.valid_vertical_moves(board, x, y, attacks_only=attacks_only))
        moves.extend(self.valid_diagonal_moves(board, x, y, attacks_only=attacks_only))

        return moves
        
class pawn(piece):

    def __init__(self, color) -> None:
        super().__init__(color)

    def find_moves(self, board, x, y, attacks_only = False):
        moves = []

        if self.color == 'w':
            direction = -1
            start = 7
        else:
            direction = 1
            start = 2
        if y == start:
            forward_lim = 2
        else:
            forward_lim = 1
        
        def forward_right_func(x, y, i):
            return x + i, (y + i*direction)
        def forward_left_func(x, y, i):
            return x - i, (y + i*direction)
        
        moves.extend(self.valid_moves_helper(board, x, y, forward_right_func, 1, pawn_take = True, attacks_only=attacks_only))
        moves.extend(self.valid_moves_helper(board, x, y, forward_left_func, 1, pawn_take = True, attacks_only=attacks_only))

        # for checking where the pawn protects
        if attacks_only == False:

            def forward(x, y, i):
                return x, (y + i*direction)

            moves.extend(self.valid_moves_helper(board, x, y, forward, forward_lim))

        # en pessant 
        last_x1, last_y1, last_x2, last_y2 = game.last_move
        if isinstance(board.get_square(last_x2,last_y2), pawn) and abs(last_y2-last_y1) == 2 and last_y2 == y:
            moves.extend([(last_x1, int((last_y1 + last_y2)/2))])
        return moves
        
class rook(piece):

    def __init__(self, color) -> None:
        super().__init__(color)
        self.moved = False

    def find_moves(self, board, x, y, attacks_only=False):
        moves = []

        moves.extend(self.valid_horizontal_moves(board, x, y, attacks_only=attacks_only))
        moves.extend(self.valid_vertical_moves(board, x, y, attacks_only=attacks_only))

        return moves
        
class bishop(piece):

    def __init__(self, color) -> None:
        super().__init__(color)

    def find_moves(self, board, x, y, attacks_only=False):
        moves = []

        moves.extend(self.valid_diagonal_moves(board, x, y, attacks_only=attacks_only))

        return moves

In [279]:
class chess_board(pd.DataFrame):
    
    def __init__(self, df = None) -> None:
        
        if isinstance(df, pd.DataFrame): 
            transfer = df.to_dict()
            super().__init__(transfer)

        else:
            super().__init__(columns=[1,2,3,4,5,6,7,8], index=reversed(range(8)))
            self.rename({0:8, 1:7, 2:6, 3:5, 5:3, 6:2, 7:1}, inplace=True)
            self.loc[8] = [rook('w'), knight('w'), bishop('w'), queen('w'), king('w'), bishop('w'), knight('w'), rook('w')]
            self.loc[7] = [pawn('w')]
            # board.loc[3] = [rook('w'), '',pawn('b'),'',king('w'),'', '', rook('w')]
            self.loc[2] = [pawn('b')]
            self.loc[1] = [rook('b'), knight('b'), bishop('b'), queen('b'), king('b'), bishop('b'), knight('b'), rook('b')]

            self.fillna('', inplace=True)

    def get_square(self, x, y):
        if x<1 or x>8 or y<1 or y>8:
            return None
        return self.loc[y][x]
    
    def get_moves(self, x, y, game, show = False):
        piece = self.get_square(x,y)
        print('piece = ', piece)
        if piece == None or piece == '':
            return []
        else:
            moves = piece.find_moves(self, x, y)

        for x2,y2 in moves:
            if self.check_move_for_checks(game, x, y, x2, y2) == False:
                moves.remove((x2, y2))

        if show: 
            board_copy = self.copy()
            for x,y in moves:
                board_copy.loc[y,x] = 'X' + str(board_copy.loc[y][x])

            display(board_copy)

        return moves
    
    def move(self, game, x1, y1, x2, y2, show = False):

        moves = self.get_moves(x1,y1, game)
        print(moves)
        if (x2,y2) not in moves:
            print('invalid move')
            return False
        else:
            print('valid move')
            self.move_piece(x1, y1, x2, y2)
            

        if isinstance(piece, (rook, king)):
            piece.moved = True
        
        if show:
            board_copy = self.copy()
            display(board_copy)
        
        return True

    def move_piece(self, x1, y1, x2, y2):
        """Function to physically move the piece on the board (regardless if legal or not)"""
        piece = self.get_square(x1,y1)
        if isinstance(piece, pawn) and ((piece.color == 'w' and y2 == 1) or (piece.color == 'b' and y2 == 8)):
            self.loc[y2,x2] = queen(piece.color)
            self.loc[y1,x1] = ''
        # en pessant
        elif isinstance(piece, pawn) and (x2 != x1) and self.get_square(x2,y2) =='':
            self.loc[y2,x2] = self.loc[y1,x1] 
            self.loc[y1,x1] = ''
            self.loc[y1,x2] = ''
        else:
            self.loc[y2,x2] = self.loc[y1,x1] 
            self.loc[y1,x1] = ''

            # castling
            if isinstance(piece, king) and abs(x2-x1) == 2:
                if x2 == 7: 
                    self.loc[y2,6] = self.loc[y1,8]
                    self.loc[y1,8] = ''
                elif x2 == 3:
                    self.loc[y2,4] = self.loc[y1,1]
                    self.loc[y1,1] = ''
    
    def get_attacks(self, color):
        color_attacks = set()
        for y, row in self.iterrows():
            for x,piece in row.items():
                if piece != '' and piece.color == color:
                    print('checking attacks '+ str(x)+ str(y)+str(piece))

                    attacks = piece.find_moves(self,x,y, attacks_only=True)
                    print(attacks)
                    color_attacks.update(attacks)

        return color_attacks
    
    def check_move_for_checks(self, game, x1, y1, x2, y2):
        """checks to make sure king is not put in check by a move"""
        color = self.get_square(x1,y1).color
        board_copy = self.board_copy()
        board_copy.move_piece(x1, y1, x2, y2)
        print(board_copy)

        if color == 'w': 
            look_for_checks = board_copy.get_attacks(game, 'b')
        else:
            look_for_checks = board_copy.get_attacks(game, 'w')
        print("move checking: ",x2,y2)
        print('look_for_checks', look_for_checks)
        for x,y in look_for_checks:
            piece = self.get_square(x,y)
            
            if isinstance(piece, king) and piece.color == color:
                return False
            
        return True
    
    def board_copy(self):
        new_board_df = self.copy(deep=True)
        new_board = chess_board(new_board_df)

        return new_board


board = chess_board()
print(board.board_copy())

   1  2  3  4  5  6  7  8
1  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖
2  ♙  ♙  ♙  ♙  ♙  ♙  ♙  ♙
3                        
4                        
5                        
6                        
7  ♟  ♟  ♟  ♟  ♟  ♟  ♟  ♟
8  ♜  ♞  ♝  ♛  ♚  ♝  ♞  ♜


In [280]:
class game:

    def __init__(self) -> None:

        self.board = chess_board()
        self.status = 'none'
        self.turn = 'w'
        self.last_move = -99, -99, -99, -99

    def get_moves(self, x, y, show = False):
        return self.board.get_moves(x, y, self, show)

    def move(self, x1, y1, x2, y2, show = False):
        piece = self.board.get_square(x1, y1)
        if piece != '' and piece.color != self.turn:
            print('Wrong turn')
            return
        
        # try to move piece
        success = self.board.move(self, x1, y1, x2, y2, show)

        # look for new checks if move was completed
        if success:

            if self.turn == 'w':
                self.turn = 'b'
                look_for_checks = self.board.get_attacks(self, 'w')
            else:
                self.turn = 'w'
                look_for_checks = self.board.get_attacks(self, 'b')
            self.last_move = (x1,y1,x2,y2)

            for x,y in look_for_checks:
                piece = self.board.get_square(x,y)
                if isinstance(piece, king) and piece.color == self.turn:
                    self.status = 'check'
    
    # def checkout_move(self, x1, y1, x2, y2):
    #     board_copy = self.board.copy()
    #     board_copy.move(x1, y1, x2, y2)


In [281]:
piece_dict = {'b_king':'♔', 
                'b_queen': '♕', 
                'b_rook': '♖', 
                'b_bishop':'♗', 
                'b_knight' : '♘', 
                'b_pawn':'♙', 
                'w_king': '♚',
                'w_queen': '♛',
                'w_rook':'♜', 
                'w_bishop':'♝', 
                'w_knight': '♞',
                'w_pawn' : '♟' ,
                '':''}
        
def switch_piece_format(square):
    return piece_dict[str(square)]

In [282]:
K1 = knight('b')

In [283]:
type(new_game.get_square(6,4))

AttributeError: 'game' object has no attribute 'get_square'

In [284]:
new_game = game()

In [285]:
new_game.board

,1,2,3,4,5,6,7,8
1,♖,♘,♗,♕,♔,♗,♘,♖
2,♙,♙,♙,♙,♙,♙,♙,♙
3,,,,,,,,
4,,,,,,,,
5,,,,,,,,
6,,,,,,,,
7,♟,♟,♟,♟,♟,♟,♟,♟
8,♜,♞,♝,♛,♚,♝,♞,♜


In [286]:
print(new_game.board.get_attacks(new_game, 'b'))

checking attacks 11♖
[(2, 1), (1, 2)]
checking attacks 21♘
[(3, 3), (4, 2), (1, 3)]
checking attacks 31♗
[(4, 2), (2, 2)]
checking attacks 41♕
[(5, 1), (3, 1), (4, 2), (5, 2), (3, 2)]
checking attacks 51♔
[(6, 1), (4, 1), (5, 2), (6, 2), (4, 2)]
checking attacks 61♗
[(7, 2), (5, 2)]
checking attacks 71♘
[(8, 3), (5, 2), (6, 3)]
checking attacks 81♖
[(7, 1), (8, 2)]
checking attacks 12♙
[(2, 3)]
checking attacks 22♙
[(3, 3), (1, 3)]
checking attacks 32♙
[(4, 3), (2, 3)]
checking attacks 42♙
[(5, 3), (3, 3)]
checking attacks 52♙
[(6, 3), (4, 3)]
checking attacks 62♙
[(7, 3), (5, 3)]
checking attacks 72♙
[(8, 3), (6, 3)]
checking attacks 82♙
[(7, 3)]
{(4, 3), (3, 1), (5, 1), (8, 3), (2, 2), (1, 3), (6, 2), (7, 1), (4, 2), (3, 3), (5, 3), (8, 2), (1, 2), (2, 1), (6, 1), (7, 3), (3, 2), (4, 1), (5, 2), (2, 3), (7, 2), (6, 3)}


In [287]:
# new_game.get_moves(5,3, show = True)

In [288]:
new_game.move(4,7,4,5,True)

piece =  ♟
   1  2  3  4  5  6  7  8
1  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖
2  ♙  ♙  ♙  ♙  ♙  ♙  ♙  ♙
3                        
4                        
5                        
6           ♟            
7  ♟  ♟  ♟     ♟  ♟  ♟  ♟
8  ♜  ♞  ♝  ♛  ♚  ♝  ♞  ♜
checking attacks 11♖
[(2, 1), (1, 2)]
checking attacks 21♘
[(3, 3), (4, 2), (1, 3)]
checking attacks 31♗
[(4, 2), (2, 2)]
checking attacks 41♕
[(5, 1), (3, 1), (4, 2), (5, 2), (3, 2)]
checking attacks 51♔
[(6, 1), (4, 1), (5, 2), (6, 2), (4, 2)]
checking attacks 61♗
[(7, 2), (5, 2)]
checking attacks 71♘
[(8, 3), (5, 2), (6, 3)]
checking attacks 81♖
[(7, 1), (8, 2)]
checking attacks 12♙
[(2, 3)]
checking attacks 22♙
[(3, 3), (1, 3)]
checking attacks 32♙
[(4, 3), (2, 3)]
checking attacks 42♙
[(5, 3), (3, 3)]
checking attacks 52♙
[(6, 3), (4, 3)]
checking attacks 62♙
[(7, 3), (5, 3)]
checking attacks 72♙
[(8, 3), (6, 3)]
checking attacks 82♙
[(7, 3)]
move checking:  4 6
look_for_checks {(4, 3), (3, 1), (5, 1), (8, 3), (2, 2), (1, 3), (6, 2), (7, 1)

,1,2,3,4,5,6,7,8
1,♖,♘,♗,♕,♔,♗,♘,♖
2,♙,♙,♙,♙,♙,♙,♙,♙
3,,,,,,,,
4,,,,,,,,
5,,,,♟,,,,
6,,,,,,,,
7,♟,♟,♟,,♟,♟,♟,♟
8,♜,♞,♝,♛,♚,♝,♞,♜


checking attacks 45♟
[(5, 4), (3, 4)]
checking attacks 17♟
[(2, 6)]
checking attacks 27♟
[(3, 6), (1, 6)]
checking attacks 37♟
[(4, 6), (2, 6)]
checking attacks 57♟
[(6, 6), (4, 6)]
checking attacks 67♟
[(7, 6), (5, 6)]
checking attacks 77♟
[(8, 6), (6, 6)]
checking attacks 87♟
[(7, 6)]
checking attacks 18♜
[(2, 8), (1, 7)]
checking attacks 28♞
[(3, 6), (4, 7), (1, 6)]
checking attacks 38♝
[(4, 7), (5, 6), (6, 5), (7, 4), (8, 3), (2, 7)]
checking attacks 48♛
[(5, 8), (3, 8), (4, 7), (4, 6), (4, 5), (5, 7), (3, 7)]
checking attacks 58♚
[(6, 8), (4, 8), (5, 7), (6, 7), (4, 7)]
checking attacks 68♝
[(7, 7), (5, 7)]
checking attacks 78♞
[(8, 6), (6, 6), (5, 7)]
checking attacks 88♜
[(7, 8), (8, 7)]


In [289]:
new_game.move(4,2,4,4,True)

piece =  ♙
   1  2  3  4  5  6  7  8
1  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖
2  ♙  ♙  ♙     ♙  ♙  ♙  ♙
3           ♙            
4                        
5           ♟            
6                        
7  ♟  ♟  ♟     ♟  ♟  ♟  ♟
8  ♜  ♞  ♝  ♛  ♚  ♝  ♞  ♜
checking attacks 45♟
[(5, 4), (3, 4), (4, 6)]
checking attacks 17♟
[(2, 6)]
checking attacks 27♟
[(3, 6), (1, 6)]
checking attacks 37♟
[(4, 6), (2, 6)]
checking attacks 57♟
[(6, 6), (4, 6)]
checking attacks 67♟
[(7, 6), (5, 6)]
checking attacks 77♟
[(8, 6), (6, 6)]
checking attacks 87♟
[(7, 6)]
checking attacks 18♜
[(2, 8), (1, 7)]
checking attacks 28♞
[(3, 6), (4, 7), (1, 6)]
checking attacks 38♝
[(4, 7), (5, 6), (6, 5), (7, 4), (8, 3), (2, 7)]
checking attacks 48♛
[(5, 8), (3, 8), (4, 7), (4, 6), (4, 5), (5, 7), (3, 7)]
checking attacks 58♚
[(6, 8), (4, 8), (5, 7), (6, 7), (4, 7)]
checking attacks 68♝
[(7, 7), (5, 7)]
checking attacks 78♞
[(8, 6), (6, 6), (5, 7)]
checking attacks 88♜
[(7, 8), (8, 7)]
move checking:  4 3
look_for_checks {(3, 4)

,1,2,3,4,5,6,7,8
1,♖,♘,♗,♕,♔,♗,♘,♖
2,♙,♙,♙,,♙,♙,♙,♙
3,,,,,,,,
4,,,,♙,,,,
5,,,,♟,,,,
6,,,,,,,,
7,♟,♟,♟,,♟,♟,♟,♟
8,♜,♞,♝,♛,♚,♝,♞,♜


checking attacks 11♖
[(2, 1), (1, 2)]
checking attacks 21♘
[(3, 3), (4, 2), (1, 3)]
checking attacks 31♗
[(4, 2), (5, 3), (6, 4), (7, 5), (8, 6), (2, 2)]
checking attacks 41♕
[(5, 1), (3, 1), (4, 2), (4, 3), (4, 4), (5, 2), (3, 2)]
checking attacks 51♔
[(6, 1), (4, 1), (5, 2), (6, 2), (4, 2)]
checking attacks 61♗
[(7, 2), (5, 2)]
checking attacks 71♘
[(8, 3), (5, 2), (6, 3)]
checking attacks 81♖
[(7, 1), (8, 2)]
checking attacks 12♙
[(2, 3)]
checking attacks 22♙
[(3, 3), (1, 3)]
checking attacks 32♙
[(4, 3), (2, 3)]
checking attacks 52♙
[(6, 3), (4, 3)]
checking attacks 62♙
[(7, 3), (5, 3)]
checking attacks 72♙
[(8, 3), (6, 3)]
checking attacks 82♙
[(7, 3)]
checking attacks 44♙
[(5, 5), (3, 5)]


In [290]:
new_game.last_move

(4, 2, 4, 4)

In [291]:
new_game.move(4,7,7,4,True)

piece =  
[]
invalid move


In [292]:
new_game.move(2,8,3,6,True)

piece =  ♞
   1  2  3  4  5  6  7  8
1  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖
2  ♙  ♙  ♙     ♙  ♙  ♙  ♙
3                        
4           ♙            
5           ♟            
6        ♞               
7  ♟  ♟  ♟     ♟  ♟  ♟  ♟
8  ♜     ♝  ♛  ♚  ♝  ♞  ♜
checking attacks 11♖
[(2, 1), (1, 2)]
checking attacks 21♘
[(3, 3), (4, 2), (1, 3)]
checking attacks 31♗
[(4, 2), (5, 3), (6, 4), (7, 5), (8, 6), (2, 2)]
checking attacks 41♕
[(5, 1), (3, 1), (4, 2), (4, 3), (4, 4), (5, 2), (3, 2)]
checking attacks 51♔
[(6, 1), (4, 1), (5, 2), (6, 2), (4, 2)]
checking attacks 61♗
[(7, 2), (5, 2)]
checking attacks 71♘
[(8, 3), (5, 2), (6, 3)]
checking attacks 81♖
[(7, 1), (8, 2)]
checking attacks 12♙
[(2, 3)]
checking attacks 22♙
[(3, 3), (1, 3)]
checking attacks 32♙
[(4, 3), (2, 3)]
checking attacks 52♙
[(6, 3), (4, 3)]
checking attacks 62♙
[(7, 3), (5, 3)]
checking attacks 72♙
[(8, 3), (6, 3)]
checking attacks 82♙
[(7, 3)]
checking attacks 44♙
[(5, 5), (3, 5), (4, 3)]
move checking:  3 6
look_for_checks {(4, 3)

,1,2,3,4,5,6,7,8
1,♖,♘,♗,♕,♔,♗,♘,♖
2,♙,♙,♙,,♙,♙,♙,♙
3,,,,,,,,
4,,,,♙,,,,
5,,,,♟,,,,
6,,,♞,,,,,
7,♟,♟,♟,,♟,♟,♟,♟
8,♜,,♝,♛,♚,♝,♞,♜


checking attacks 45♟
[(5, 4), (3, 4)]
checking attacks 36♞
[(4, 8), (5, 7), (4, 4), (1, 7), (2, 8), (5, 5), (2, 4), (1, 5)]
checking attacks 17♟
[(2, 6)]
checking attacks 27♟
[(3, 6), (1, 6)]
checking attacks 37♟
[(4, 6), (2, 6)]
checking attacks 57♟
[(6, 6), (4, 6)]
checking attacks 67♟
[(7, 6), (5, 6)]
checking attacks 77♟
[(8, 6), (6, 6)]
checking attacks 87♟
[(7, 6)]
checking attacks 18♜
[(2, 8), (3, 8), (1, 7)]
checking attacks 38♝
[(4, 7), (5, 6), (6, 5), (7, 4), (8, 3), (2, 7)]
checking attacks 48♛
[(5, 8), (3, 8), (4, 7), (4, 6), (4, 5), (5, 7), (3, 7)]
checking attacks 58♚
[(6, 8), (4, 8), (5, 7), (6, 7), (4, 7)]
checking attacks 68♝
[(7, 7), (5, 7)]
checking attacks 78♞
[(8, 6), (6, 6), (5, 7)]
checking attacks 88♜
[(7, 8), (8, 7)]


In [293]:
new_game.move(5,2,5,4,True)

piece =  ♙
   1  2  3  4  5  6  7  8
1  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖
2  ♙  ♙  ♙        ♙  ♙  ♙
3              ♙         
4           ♙            
5           ♟            
6        ♞               
7  ♟  ♟  ♟     ♟  ♟  ♟  ♟
8  ♜     ♝  ♛  ♚  ♝  ♞  ♜
checking attacks 45♟
[(5, 4), (3, 4)]
checking attacks 36♞
[(4, 8), (5, 7), (4, 4), (1, 7), (2, 8), (5, 5), (2, 4), (1, 5)]
checking attacks 17♟
[(2, 6)]
checking attacks 27♟
[(3, 6), (1, 6)]
checking attacks 37♟
[(4, 6), (2, 6)]
checking attacks 57♟
[(6, 6), (4, 6)]
checking attacks 67♟
[(7, 6), (5, 6)]
checking attacks 77♟
[(8, 6), (6, 6)]
checking attacks 87♟
[(7, 6)]
checking attacks 18♜
[(2, 8), (3, 8), (1, 7)]
checking attacks 38♝
[(4, 7), (5, 6), (6, 5), (7, 4), (8, 3), (2, 7)]
checking attacks 48♛
[(5, 8), (3, 8), (4, 7), (4, 6), (4, 5), (5, 7), (3, 7)]
checking attacks 58♚
[(6, 8), (4, 8), (5, 7), (6, 7), (4, 7)]
checking attacks 68♝
[(7, 7), (5, 7)]
checking attacks 78♞
[(8, 6), (6, 6), (5, 7)]
checking attacks 88♜
[(7, 8), (8, 7)]
mov

,1,2,3,4,5,6,7,8
1,♖,♘,♗,♕,♔,♗,♘,♖
2,♙,♙,♙,,,♙,♙,♙
3,,,,,,,,
4,,,,♙,♙,,,
5,,,,♟,,,,
6,,,♞,,,,,
7,♟,♟,♟,,♟,♟,♟,♟
8,♜,,♝,♛,♚,♝,♞,♜


checking attacks 11♖
[(2, 1), (1, 2)]
checking attacks 21♘
[(3, 3), (4, 2), (1, 3)]
checking attacks 31♗
[(4, 2), (5, 3), (6, 4), (7, 5), (8, 6), (2, 2)]
checking attacks 41♕
[(5, 1), (3, 1), (4, 2), (4, 3), (4, 4), (5, 2), (6, 3), (7, 4), (8, 5), (3, 2)]
checking attacks 51♔
[(6, 1), (4, 1), (5, 2), (6, 2), (4, 2)]
checking attacks 61♗
[(7, 2), (5, 2), (4, 3), (3, 4), (2, 5), (1, 6)]
checking attacks 71♘
[(8, 3), (5, 2), (6, 3)]
checking attacks 81♖
[(7, 1), (8, 2)]
checking attacks 12♙
[(2, 3)]
checking attacks 22♙
[(3, 3), (1, 3)]
checking attacks 32♙
[(4, 3), (2, 3)]
checking attacks 62♙
[(7, 3), (5, 3)]
checking attacks 72♙
[(8, 3), (6, 3)]
checking attacks 82♙
[(7, 3)]
checking attacks 44♙
[(5, 5), (3, 5)]
checking attacks 54♙
[(6, 5), (4, 5)]


In [294]:
new_game.move(3,6,4,4,True)

piece =  ♞
   1  2  3  4  5  6  7  8
1  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖
2  ♙  ♙  ♙        ♙  ♙  ♙
3                        
4           ♞  ♙         
5           ♟            
6                        
7  ♟  ♟  ♟     ♟  ♟  ♟  ♟
8  ♜     ♝  ♛  ♚  ♝  ♞  ♜
checking attacks 11♖
[(2, 1), (1, 2)]
checking attacks 21♘
[(3, 3), (4, 2), (1, 3)]
checking attacks 31♗
[(4, 2), (5, 3), (6, 4), (7, 5), (8, 6), (2, 2)]
checking attacks 41♕
[(5, 1), (3, 1), (4, 2), (4, 3), (4, 4), (5, 2), (6, 3), (7, 4), (8, 5), (3, 2)]
checking attacks 51♔
[(6, 1), (4, 1), (5, 2), (6, 2), (4, 2)]
checking attacks 61♗
[(7, 2), (5, 2), (4, 3), (3, 4), (2, 5), (1, 6)]
checking attacks 71♘
[(8, 3), (5, 2), (6, 3)]
checking attacks 81♖
[(7, 1), (8, 2)]
checking attacks 12♙
[(2, 3)]
checking attacks 22♙
[(3, 3), (1, 3)]
checking attacks 32♙
[(4, 3), (2, 3)]
checking attacks 62♙
[(7, 3), (5, 3)]
checking attacks 72♙
[(8, 3), (6, 3)]
checking attacks 82♙
[(7, 3)]
checking attacks 54♙
[(6, 5), (4, 5), (5, 3)]
move checking:  4 4
look_

,1,2,3,4,5,6,7,8
1,♖,♘,♗,♕,♔,♗,♘,♖
2,♙,♙,♙,,,♙,♙,♙
3,,,,,,,,
4,,,,♞,♙,,,
5,,,,♟,,,,
6,,,,,,,,
7,♟,♟,♟,,♟,♟,♟,♟
8,♜,,♝,♛,♚,♝,♞,♜


checking attacks 44♞
[(5, 6), (6, 5), (5, 2), (2, 5), (3, 6), (6, 3), (3, 2), (2, 3)]
checking attacks 45♟
[(5, 4), (3, 4)]
checking attacks 17♟
[(2, 6)]
checking attacks 27♟
[(3, 6), (1, 6)]
checking attacks 37♟
[(4, 6), (2, 6)]
checking attacks 57♟
[(6, 6), (4, 6)]
checking attacks 67♟
[(7, 6), (5, 6)]
checking attacks 77♟
[(8, 6), (6, 6)]
checking attacks 87♟
[(7, 6)]
checking attacks 18♜
[(2, 8), (3, 8), (1, 7)]
checking attacks 38♝
[(4, 7), (5, 6), (6, 5), (7, 4), (8, 3), (2, 7)]
checking attacks 48♛
[(5, 8), (3, 8), (4, 7), (4, 6), (4, 5), (5, 7), (3, 7)]
checking attacks 58♚
[(6, 8), (4, 8), (5, 7), (6, 7), (4, 7)]
checking attacks 68♝
[(7, 7), (5, 7)]
checking attacks 78♞
[(8, 6), (6, 6), (5, 7)]
checking attacks 88♜
[(7, 8), (8, 7)]


In [295]:
new_game.move(5,4,5,5,True)

piece =  ♙
   1  2  3  4  5  6  7  8
1  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖
2  ♙  ♙  ♙        ♙  ♙  ♙
3                        
4           ♞            
5           ♙            
6                        
7  ♟  ♟  ♟     ♟  ♟  ♟  ♟
8  ♜     ♝  ♛  ♚  ♝  ♞  ♜
checking attacks 44♞
[(5, 6), (6, 5), (5, 2), (2, 5), (3, 6), (6, 3), (3, 2), (2, 3)]
checking attacks 17♟
[(2, 6)]
checking attacks 27♟
[(3, 6), (1, 6)]
checking attacks 37♟
[(4, 6), (2, 6)]
checking attacks 57♟
[(6, 6), (4, 6)]
checking attacks 67♟
[(7, 6), (5, 6)]
checking attacks 77♟
[(8, 6), (6, 6)]
checking attacks 87♟
[(7, 6)]
checking attacks 18♜
[(2, 8), (3, 8), (1, 7)]
checking attacks 38♝
[(4, 7), (5, 6), (6, 5), (7, 4), (8, 3), (2, 7)]
checking attacks 48♛
[(5, 8), (3, 8), (4, 7), (4, 6), (4, 5), (5, 7), (3, 7)]
checking attacks 58♚
[(6, 8), (4, 8), (5, 7), (6, 7), (4, 7)]
checking attacks 68♝
[(7, 7), (5, 7)]
checking attacks 78♞
[(8, 6), (6, 6), (5, 7)]
checking attacks 88♜
[(7, 8), (8, 7)]
move checking:  4 5
look_for_checks {(3, 

,1,2,3,4,5,6,7,8
1,♖,♘,♗,♕,♔,♗,♘,♖
2,♙,♙,♙,,,♙,♙,♙
3,,,,,,,,
4,,,,♞,,,,
5,,,,♟,♙,,,
6,,,,,,,,
7,♟,♟,♟,,♟,♟,♟,♟
8,♜,,♝,♛,♚,♝,♞,♜


checking attacks 11♖
[(2, 1), (1, 2)]
checking attacks 21♘
[(3, 3), (4, 2), (1, 3)]
checking attacks 31♗
[(4, 2), (5, 3), (6, 4), (7, 5), (8, 6), (2, 2)]
checking attacks 41♕
[(5, 1), (3, 1), (4, 2), (4, 3), (4, 4), (5, 2), (6, 3), (7, 4), (8, 5), (3, 2)]
checking attacks 51♔
[(6, 1), (4, 1), (5, 2), (6, 2), (4, 2)]
checking attacks 61♗
[(7, 2), (5, 2), (4, 3), (3, 4), (2, 5), (1, 6)]
checking attacks 71♘
[(8, 3), (5, 2), (6, 3)]
checking attacks 81♖
[(7, 1), (8, 2)]
checking attacks 12♙
[(2, 3)]
checking attacks 22♙
[(3, 3), (1, 3)]
checking attacks 32♙
[(4, 3), (2, 3)]
checking attacks 62♙
[(7, 3), (5, 3)]
checking attacks 72♙
[(8, 3), (6, 3)]
checking attacks 82♙
[(7, 3)]
checking attacks 55♙
[(6, 6), (4, 6)]


In [296]:
new_game.move(6,7,6,5,True)

piece =  ♟
   1  2  3  4  5  6  7  8
1  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖
2  ♙  ♙  ♙        ♙  ♙  ♙
3                        
4           ♞            
5           ♟  ♙         
6                 ♟      
7  ♟  ♟  ♟     ♟     ♟  ♟
8  ♜     ♝  ♛  ♚  ♝  ♞  ♜
checking attacks 11♖
[(2, 1), (1, 2)]
checking attacks 21♘
[(3, 3), (4, 2), (1, 3)]
checking attacks 31♗
[(4, 2), (5, 3), (6, 4), (7, 5), (8, 6), (2, 2)]
checking attacks 41♕
[(5, 1), (3, 1), (4, 2), (4, 3), (4, 4), (5, 2), (6, 3), (7, 4), (8, 5), (3, 2)]
checking attacks 51♔
[(6, 1), (4, 1), (5, 2), (6, 2), (4, 2)]
checking attacks 61♗
[(7, 2), (5, 2), (4, 3), (3, 4), (2, 5), (1, 6)]
checking attacks 71♘
[(8, 3), (5, 2), (6, 3)]
checking attacks 81♖
[(7, 1), (8, 2)]
checking attacks 12♙
[(2, 3)]
checking attacks 22♙
[(3, 3), (1, 3)]
checking attacks 32♙
[(4, 3), (2, 3)]
checking attacks 62♙
[(7, 3), (5, 3)]
checking attacks 72♙
[(8, 3), (6, 3)]
checking attacks 82♙
[(7, 3)]
checking attacks 55♙
[(6, 6), (4, 6)]
move checking:  6 6
look_for_chec

,1,2,3,4,5,6,7,8
1,♖,♘,♗,♕,♔,♗,♘,♖
2,♙,♙,♙,,,♙,♙,♙
3,,,,,,,,
4,,,,♞,,,,
5,,,,♟,♙,♟,,
6,,,,,,,,
7,♟,♟,♟,,♟,,♟,♟
8,♜,,♝,♛,♚,♝,♞,♜


checking attacks 44♞
[(5, 6), (6, 5), (5, 2), (2, 5), (3, 6), (6, 3), (3, 2), (2, 3)]
checking attacks 45♟
[(5, 4), (3, 4)]
checking attacks 65♟
[(7, 4), (5, 4)]
checking attacks 17♟
[(2, 6)]
checking attacks 27♟
[(3, 6), (1, 6)]
checking attacks 37♟
[(4, 6), (2, 6)]
checking attacks 57♟
[(6, 6), (4, 6)]
checking attacks 77♟
[(8, 6), (6, 6)]
checking attacks 87♟
[(7, 6)]
checking attacks 18♜
[(2, 8), (3, 8), (1, 7)]
checking attacks 38♝
[(4, 7), (5, 6), (6, 5), (2, 7)]
checking attacks 48♛
[(5, 8), (3, 8), (4, 7), (4, 6), (4, 5), (5, 7), (3, 7)]
checking attacks 58♚
[(6, 8), (4, 8), (5, 7), (6, 7), (4, 7)]
checking attacks 68♝
[(7, 7), (5, 7)]
checking attacks 78♞
[(8, 6), (6, 6), (5, 7)]
checking attacks 88♜
[(7, 8), (8, 7)]


In [297]:
print(new_game.last_move)
new_game.move(5,5,6,6,True)

(6, 7, 6, 5)
piece =  ♙
   1  2  3  4  5  6  7  8
1  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖
2  ♙  ♙  ♙        ♙  ♙  ♙
3                        
4           ♞            
5           ♟     ♟      
6              ♙         
7  ♟  ♟  ♟     ♟     ♟  ♟
8  ♜     ♝  ♛  ♚  ♝  ♞  ♜
checking attacks 44♞
[(5, 6), (6, 5), (5, 2), (2, 5), (3, 6), (6, 3), (3, 2), (2, 3)]
checking attacks 45♟
[(5, 4), (3, 4), (6, 6)]
checking attacks 65♟
[(7, 4), (5, 4), (6, 6)]
checking attacks 17♟
[(2, 6)]
checking attacks 27♟
[(3, 6), (1, 6)]
checking attacks 37♟
[(4, 6), (2, 6)]
checking attacks 57♟
[(6, 6), (4, 6)]
checking attacks 77♟
[(8, 6), (6, 6)]
checking attacks 87♟
[(7, 6)]
checking attacks 18♜
[(2, 8), (3, 8), (1, 7)]
checking attacks 38♝
[(4, 7), (5, 6), (6, 5), (2, 7)]
checking attacks 48♛
[(5, 8), (3, 8), (4, 7), (4, 6), (4, 5), (5, 7), (3, 7)]
checking attacks 58♚
[(6, 8), (4, 8), (5, 7), (6, 7), (4, 7)]
checking attacks 68♝
[(7, 7), (5, 7)]
checking attacks 78♞
[(8, 6), (6, 6), (5, 7)]
checking attacks 88♜
[(7, 8)

,1,2,3,4,5,6,7,8
1,♖,♘,♗,♕,♔,♗,♘,♖
2,♙,♙,♙,,,♙,♙,♙
3,,,,,,,,
4,,,,♞,,,,
5,,,,♟,,,,
6,,,,,,♙,,
7,♟,♟,♟,,♟,,♟,♟
8,♜,,♝,♛,♚,♝,♞,♜


checking attacks 11♖
[(2, 1), (1, 2)]
checking attacks 21♘
[(3, 3), (4, 2), (1, 3)]
checking attacks 31♗
[(4, 2), (5, 3), (6, 4), (7, 5), (8, 6), (2, 2)]
checking attacks 41♕
[(5, 1), (3, 1), (4, 2), (4, 3), (4, 4), (5, 2), (6, 3), (7, 4), (8, 5), (3, 2)]
checking attacks 51♔
[(6, 1), (4, 1), (5, 2), (6, 2), (4, 2)]
checking attacks 61♗
[(7, 2), (5, 2), (4, 3), (3, 4), (2, 5), (1, 6)]
checking attacks 71♘
[(8, 3), (5, 2), (6, 3)]
checking attacks 81♖
[(7, 1), (8, 2)]
checking attacks 12♙
[(2, 3)]
checking attacks 22♙
[(3, 3), (1, 3)]
checking attacks 32♙
[(4, 3), (2, 3)]
checking attacks 62♙
[(7, 3), (5, 3)]
checking attacks 72♙
[(8, 3), (6, 3)]
checking attacks 82♙
[(7, 3)]
checking attacks 66♙
[(7, 7), (5, 7)]


In [298]:
new_game.move(5,7,6,6,True)
new_game.move(4,1,5,2,True)
new_game.move(6,5,6,4,True)
new_game.get_moves(5,1, True)

piece =  ♟
   1  2  3  4  5  6  7  8
1  ♖  ♘  ♗  ♕  ♔  ♗  ♘  ♖
2  ♙  ♙  ♙        ♙  ♙  ♙
3                        
4           ♞            
5           ♟            
6                 ♟      
7  ♟  ♟  ♟           ♟  ♟
8  ♜     ♝  ♛  ♚  ♝  ♞  ♜
checking attacks 11♖
[(2, 1), (1, 2)]
checking attacks 21♘
[(3, 3), (4, 2), (1, 3)]
checking attacks 31♗
[(4, 2), (5, 3), (6, 4), (7, 5), (8, 6), (2, 2)]
checking attacks 41♕
[(5, 1), (3, 1), (4, 2), (4, 3), (4, 4), (5, 2), (6, 3), (7, 4), (8, 5), (3, 2)]
checking attacks 51♔
[(6, 1), (4, 1), (5, 2), (6, 2), (4, 2)]
checking attacks 61♗
[(7, 2), (5, 2), (4, 3), (3, 4), (2, 5), (1, 6)]
checking attacks 71♘
[(8, 3), (5, 2), (6, 3)]
checking attacks 81♖
[(7, 1), (8, 2)]
checking attacks 12♙
[(2, 3)]
checking attacks 22♙
[(3, 3), (1, 3)]
checking attacks 32♙
[(4, 3), (2, 3)]
checking attacks 62♙
[(7, 3), (5, 3)]
checking attacks 72♙
[(8, 3), (6, 3)]
checking attacks 82♙
[(7, 3)]
move checking:  6 6
look_for_checks {(3, 4), (4, 3), (3, 1), (5, 1), (8

,1,2,3,4,5,6,7,8
1,♖,♘,♗,♕,♔,♗,♘,♖
2,♙,♙,♙,,,♙,♙,♙
3,,,,,,,,
4,,,,♞,,,,
5,,,,♟,,,,
6,,,,,,♟,,
7,♟,♟,♟,,,,♟,♟
8,♜,,♝,♛,♚,♝,♞,♜


checking attacks 44♞
[(5, 6), (6, 5), (5, 2), (2, 5), (3, 6), (6, 3), (3, 2), (2, 3)]
checking attacks 45♟
[(5, 4), (3, 4)]
checking attacks 66♟
[(7, 5), (5, 5)]
checking attacks 17♟
[(2, 6)]
checking attacks 27♟
[(3, 6), (1, 6)]
checking attacks 37♟
[(4, 6), (2, 6)]
checking attacks 77♟
[(8, 6), (6, 6)]
checking attacks 87♟
[(7, 6)]
checking attacks 18♜
[(2, 8), (3, 8), (1, 7)]
checking attacks 38♝
[(4, 7), (5, 6), (6, 5), (7, 4), (8, 3), (2, 7)]
checking attacks 48♛
[(5, 8), (3, 8), (4, 7), (4, 6), (4, 5), (5, 7), (6, 6), (3, 7)]
checking attacks 58♚
[(6, 8), (4, 8), (5, 7), (6, 7), (4, 7)]
checking attacks 68♝
[(7, 7), (5, 7), (4, 6), (3, 5), (2, 4), (1, 3)]
checking attacks 78♞
[(8, 6), (6, 6), (5, 7)]
checking attacks 88♜
[(7, 8), (8, 7)]
piece =  ♕
   1  2  3  4  5  6  7  8
1  ♖  ♘  ♗     ♔  ♗  ♘  ♖
2  ♙  ♙  ♙  ♕     ♙  ♙  ♙
3                        
4           ♞            
5           ♟            
6                 ♟      
7  ♟  ♟  ♟           ♟  ♟
8  ♜     ♝  ♛  ♚  ♝  ♞  ♜
c

,1,2,3,4,5,6,7,8
1,♖,♘,♗,,♔,♗,♘,♖
2,♙,♙,♙,,♕,♙,♙,♙
3,,,,,,,,
4,,,,♞,,,,
5,,,,♟,,,,
6,,,,,,♟,,
7,♟,♟,♟,,,,♟,♟
8,♜,,♝,♛,♚,♝,♞,♜


checking attacks 11♖
[(2, 1), (1, 2)]
checking attacks 21♘
[(3, 3), (4, 2), (1, 3)]
checking attacks 31♗
[(4, 2), (5, 3), (6, 4), (7, 5), (8, 6), (2, 2)]
checking attacks 51♔
[(6, 1), (4, 1), (5, 2), (6, 2), (4, 2)]
checking attacks 61♗
[(7, 2), (5, 2)]
checking attacks 71♘
[(8, 3), (5, 2), (6, 3)]
checking attacks 81♖
[(7, 1), (8, 2)]
checking attacks 12♙
[(2, 3)]
checking attacks 22♙
[(3, 3), (1, 3)]
checking attacks 32♙
[(4, 3), (2, 3)]
checking attacks 52♕
[(6, 2), (4, 2), (3, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 1), (6, 3), (7, 4), (8, 5), (6, 1), (4, 3), (3, 4), (2, 5), (1, 6), (4, 1)]
checking attacks 62♙
[(7, 3), (5, 3)]
checking attacks 72♙
[(8, 3), (6, 3)]
checking attacks 82♙
[(7, 3)]
piece =  
[]
invalid move
piece =  ♔
checking attacks 44♞
[(5, 6), (6, 5), (5, 2), (2, 5), (3, 6), (6, 3), (3, 2), (2, 3)]
checking attacks 45♟
[(5, 4), (3, 4)]
checking attacks 66♟
[(7, 5), (5, 5)]
checking attacks 17♟
[(2, 6)]
checking attacks 27♟
[(3, 6), (1, 6)]
checking 

,1,2,3,4,5,6,7,8
1,♖,♘,♗,X,♔,♗,♘,♖
2,♙,♙,♙,X,♕,♙,♙,♙
3,,,,,,,,
4,,,,♞,,,,
5,,,,♟,,,,
6,,,,,,♟,,
7,♟,♟,♟,,,,♟,♟
8,♜,,♝,♛,♚,♝,♞,♜


[(4, 1), (4, 2)]

In [299]:
new_game.move(6,8,5,7,True)
# new_game.get_moves(5,2, True)
# new_game.turn

piece =  ♝
   1  2  3  4  5  6  7  8
1  ♖  ♘  ♗     ♔  ♗  ♘  ♖
2  ♙  ♙  ♙     ♕  ♙  ♙  ♙
3                        
4           ♞            
5           ♟            
6                 ♟      
7  ♟  ♟  ♟     ♝     ♟  ♟
8  ♜     ♝  ♛  ♚     ♞  ♜
checking attacks 11♖
[(2, 1), (1, 2)]
checking attacks 21♘
[(3, 3), (4, 2), (1, 3)]
checking attacks 31♗
[(4, 2), (5, 3), (6, 4), (7, 5), (8, 6), (2, 2)]
checking attacks 51♔
[(6, 1), (4, 1), (5, 2), (6, 2), (4, 2)]
checking attacks 61♗
[(7, 2), (5, 2)]
checking attacks 71♘
[(8, 3), (5, 2), (6, 3)]
checking attacks 81♖
[(7, 1), (8, 2)]
checking attacks 12♙
[(2, 3)]
checking attacks 22♙
[(3, 3), (1, 3)]
checking attacks 32♙
[(4, 3), (2, 3)]
checking attacks 52♕
[(6, 2), (4, 2), (3, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 1), (6, 3), (7, 4), (8, 5), (6, 1), (4, 3), (3, 4), (2, 5), (1, 6), (4, 1)]
checking attacks 62♙
[(7, 3), (5, 3)]
checking attacks 72♙
[(8, 3), (6, 3)]
checking attacks 82♙
[(7, 3)]
move checking:  5 7
look_for_ch

In [144]:
board = pd.DataFrame(columns=[1,2,3,4,5,6,7,8], index=reversed(range(8)))
print(board)

     1    2    3    4    5    6    7    8
7  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
6  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
5  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
3  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
